In [1]:
import pandas as pd
import numpy as np

# Read in the data with the first two lines as a multi-index and the 'Player ID' column as the index
df = pd.read_csv('./interim/nba_draft_data_combined.csv', header=[0,1], index_col=0)

# replace all 0s with NaNs
df = df.replace(0, np.nan)

df.head()

Career    Career                                                           \
Player ID Season Pk   Tm               Player       College   Yrs       G   
947.0       1996  1  PHI        Allen Iverson    Georgetown  14.0   914.0   
948.0       1996  2  TOR         Marcus Camby         UMass  17.0   973.0   
949.0       1996  3  VAN  Shareef Abdur-Rahim    California  12.0   830.0   
950.0       1996  4  MIL      Stephon Marbury  Georgia Tech  13.0   846.0   
951.0       1996  5  MIN            Ray Allen         UConn  18.0  1300.0   

Career                               ... 2021-22 2022-23                     \
Player ID       MP      PTS     TRB  ... TEAM_ID     AGE DEF_RATING  GP MIN   
947.0      37584.0  24368.0  3394.0  ...     NaN     NaN        NaN NaN NaN   
948.0      28684.0   9262.0  9513.0  ...     NaN     NaN        NaN NaN NaN   
949.0      28882.0  15028.0  6239.0  ...     NaN     NaN        NaN NaN NaN   
950.0      31891.0  16297.0  2516.0  ...     NaN     NaN        NaN NaN NaN   
951.0      46344.0  24505.0  5272.0  ...     NaN     NaN        NaN NaN NaN   

Career                                                         
Player ID NET_RATING OFF_RATING PIE TEAM_ABBREVIATION TEAM_ID  
947.0            NaN        NaN NaN                 0     NaN  
948.0            NaN        NaN NaN                 0     NaN  
949.0            NaN        NaN NaN                 0     NaN  
950.0            NaN        NaN NaN                 0     NaN  
951.0            NaN        NaN NaN                 0     NaN  

[5 rows x 265 columns]

In [2]:
# Get the position of the last career column
position_to_insert = len(df['Career'].columns)
# Get the list of columns to insert
columns_to_insert = [('Career', 'OFF_RATING'), ('Career', 'DEF_RATING'), ('Career', 'NET_RATING'), ('Career', 'PIE')]

# Insert multiple columns with NaN values
for col in columns_to_insert:
    df.insert(loc=position_to_insert, column=col, value=np.nan)

In [3]:
# create list with all seasons between 1996-97 and 2022-23
seasons = []
for i in range(1996, 2023):
    seasons.append(str(i) + '-' + str(i+1)[2:])

# for each season, insert a new column with the product of 'GP' and 'OFF_RATING'
for season in seasons:
    # find the (season, 'OFF_RATING') column as the position to insert the new column
    position_to_insert = df.columns.get_loc((season, 'OFF_RATING'))
    df.insert(loc=position_to_insert, column=(season, 'OFF_RATING_weighted'), value=np.nan)
    # fill the new column with the product of (season,'GP') and (season,'OFF_RATING')
    df[(season, 'OFF_RATING_weighted')] = df[(season, 'GP')] * df[(season, 'OFF_RATING')]
    
    position_to_insert = df.columns.get_loc((season, 'DEF_RATING'))
    df.insert(loc=position_to_insert, column=(season, 'DEF_RATING_weighted'), value=np.nan)
    df[(season, 'DEF_RATING_weighted')] = df[(season, 'GP')] * df[(season, 'DEF_RATING')]
    
    position_to_insert = df.columns.get_loc((season, 'NET_RATING'))
    df.insert(loc=position_to_insert, column=(season, 'NET_RATING_weighted'), value=np.nan)
    df[(season, 'NET_RATING_weighted')] = df[(season, 'GP')] * df[(season, 'NET_RATING')]
    
    position_to_insert = df.columns.get_loc((season, 'PIE'))
    df.insert(loc=position_to_insert, column=(season, 'PIE_weighted'), value=np.nan)
    df[(season, 'PIE_weighted')] = df[(season, 'GP')] * df[(season, 'PIE')]

/var/folders/zb/fj1ym99j70b38snb8v3_0g580000gn/T/ipykernel_12154/1738369481.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.insert(loc=position_to_insert, column=(season, 'DEF_RATING_weighted'), value=np.nan)
/var/folders/zb/fj1ym99j70b38snb8v3_0g580000gn/T/ipykernel_12154/1738369481.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.insert(loc=position_to_insert, column=(season, 'NET_RATING_weighted'), value=np.nan)
/var/folders/zb/fj1ym99j70b38snb8v3_0g580000gn/T/ipykernel_12154/1738369481.py:23: PerformanceWarnin

In [4]:
# in the ('Career', rating) column, fill the NaNs with the sum of the weighted columns divided by the ('Career', 'G') column
df[('Career', 'OFF_RATING')] = df.filter(regex='OFF_RATING_weighted').sum(axis=1) / df[('Career', 'G')]
df[('Career', 'DEF_RATING')] = df.filter(regex='DEF_RATING_weighted').sum(axis=1) / df[('Career', 'G')]
df[('Career', 'NET_RATING')] = df.filter(regex='NET_RATING_weighted').sum(axis=1) / df[('Career', 'G')]
df[('Career', 'PIE')] = df.filter(regex='PIE_weighted').sum(axis=1) / df[('Career', 'G')]
# drop the weighted columns
df = df.drop(df.filter(regex='_weighted').columns, axis=1)

In [5]:
# copy the 'Career' columns to a new dataframe
df_career = df['Career'].copy()

# export the dataframe to a csv file
df_career.to_csv('./interim/player_career_avg.csv')

df_career.head(60)


Player ID,Season,Pk,Tm,Player,College,Yrs,G,MP,PTS,TRB,...,TRB.1,AST.1,WS,WS/48,BPM,VORP,PIE,NET_RATING,DEF_RATING,OFF_RATING
947.0,1996,1,PHI,Allen Iverson,Georgetown,14.0,914.0,37584.0,24368.0,3394.0,...,3.7,6.2,99.0,0.126,3.2,49.6,0.136685,0.846389,102.536105,103.373742
948.0,1996,2,TOR,Marcus Camby,UMass,17.0,973.0,28684.0,9262.0,9513.0,...,9.8,1.9,81.6,0.137,2.3,31.3,0.126784,0.228571,102.740699,102.970298
949.0,1996,3,VAN,Shareef Abdur-Rahim,California,12.0,830.0,28882.0,15028.0,6239.0,...,7.5,2.5,71.2,0.118,1.0,21.9,0.125880,-5.022892,105.843614,100.801928
950.0,1996,4,MIL,Stephon Marbury,Georgia Tech,13.0,846.0,31891.0,16297.0,2516.0,...,3.0,7.6,77.5,0.117,1.8,30.7,0.123372,-1.780615,105.290426,103.501064
951.0,1996,5,MIN,Ray Allen,UConn,18.0,1300.0,46344.0,24505.0,5272.0,...,4.1,3.4,145.1,0.150,2.9,57.6,0.119475,3.884615,104.047000,107.940000
952.0,1996,6,BOS,Antoine Walker,Kentucky,12.0,893.0,31531.0,15647.0,6891.0,...,7.7,3.5,38.1,0.058,0.3,18.0,0.106725,-2.122172,104.507055,102.397984
953.0,1996,7,LAC,Lorenzen Wright,Memphis,13.0,778.0,18535.0,6191.0,4943.0,...,6.4,0.8,30.5,0.079,-2.6,-3.0,0.079762,-3.612725,102.883676,99.275450
954.0,1996,8,NJN,Kerry Kittles,Villanova,8.0,507.0,16929.0,7165.0,1983.0,...,3.9,2.6,44.8,0.127,2.2,18.0,0.106501,1.059763,100.949901,101.997436
955.0,1996,9,DAL,Samaki Walker,Louisville,10.0,445.0,7612.0,2376.0,2089.0,...,4.7,0.6,14.2,0.089,-2.6,-1.1,0.079793,-4.483371,101.874607,97.406517
956.0,1996,10,IND,Erick Dampier,Mississippi State,16.0,987.0,24003.0,7309.0,7005.0,...,7.1,0.8,52.7,0.105,-1.3,4.1,0.082994,0.622594,102.671530,103.291084


**Select a penalty for unbounded stats**

Drafted players who never played a game in the NBA are perceived as bad draft picks but never accumulated any stats. Since excluding them from the analysis would worsen the relative performance of other players selected at that respective draft position, the empty stats should be replaced by a "bad stat". For total points, assists and rebounds, 0 is the absolute minimum and therefore suitable as a replacement stat. However, all other stats are unbounded and don't have a definite minimum value. Using the minimum value in our data would be greatly affected by outliers which is why we opted for a quantile that can be dynamically chosen. We recommend using a quantile between 1% and 5%.

In [6]:
# TODO: Allow filtering by season range

penalty_quantile = 0.02 # TODO: Replace with widget sometime

relevant_stats = ['PTS', 'TRB', 'AST', 'WS', 'WS/48', 'BPM', 'VORP', 'PIE', 'OFF_RATING', 'DEF_RATING', 'NET_RATING']
na_fill_values = {'PTS': 0, 'TRB': 0, 'AST': 0, 'WS': df_career['WS'].quantile(penalty_quantile), 'WS/48': df_career['WS/48'].quantile(penalty_quantile), 'BPM': df_career['BPM'].quantile(penalty_quantile), 'VORP': df_career['VORP'].quantile(penalty_quantile), 'PIE': df_career['PIE'].quantile(penalty_quantile), 'OFF_RATING': df_career['OFF_RATING'].quantile(penalty_quantile), 'DEF_RATING': df_career['DEF_RATING'].quantile(1-penalty_quantile), 'NET_RATING': df_career['NET_RATING'].quantile(penalty_quantile)}

# fill the NaNs with the respective entry in the na_fill_values dict for each column
df_career_na_filled = df_career.fillna(value=na_fill_values)

df_career_na_filled.head(60)

Player ID,Season,Pk,Tm,Player,College,Yrs,G,MP,PTS,TRB,...,TRB.1,AST.1,WS,WS/48,BPM,VORP,PIE,NET_RATING,DEF_RATING,OFF_RATING
947.0,1996,1,PHI,Allen Iverson,Georgetown,14.0,914.0,37584.0,24368.0,3394.0,...,3.7,6.2,99.000,0.12600,3.20,49.6,0.136685,0.846389,102.536105,103.373742
948.0,1996,2,TOR,Marcus Camby,UMass,17.0,973.0,28684.0,9262.0,9513.0,...,9.8,1.9,81.600,0.13700,2.30,31.3,0.126784,0.228571,102.740699,102.970298
949.0,1996,3,VAN,Shareef Abdur-Rahim,California,12.0,830.0,28882.0,15028.0,6239.0,...,7.5,2.5,71.200,0.11800,1.00,21.9,0.125880,-5.022892,105.843614,100.801928
950.0,1996,4,MIL,Stephon Marbury,Georgia Tech,13.0,846.0,31891.0,16297.0,2516.0,...,3.0,7.6,77.500,0.11700,1.80,30.7,0.123372,-1.780615,105.290426,103.501064
951.0,1996,5,MIN,Ray Allen,UConn,18.0,1300.0,46344.0,24505.0,5272.0,...,4.1,3.4,145.100,0.15000,2.90,57.6,0.119475,3.884615,104.047000,107.940000
952.0,1996,6,BOS,Antoine Walker,Kentucky,12.0,893.0,31531.0,15647.0,6891.0,...,7.7,3.5,38.100,0.05800,0.30,18.0,0.106725,-2.122172,104.507055,102.397984
953.0,1996,7,LAC,Lorenzen Wright,Memphis,13.0,778.0,18535.0,6191.0,4943.0,...,6.4,0.8,30.500,0.07900,-2.60,-3.0,0.079762,-3.612725,102.883676,99.275450
954.0,1996,8,NJN,Kerry Kittles,Villanova,8.0,507.0,16929.0,7165.0,1983.0,...,3.9,2.6,44.800,0.12700,2.20,18.0,0.106501,1.059763,100.949901,101.997436
955.0,1996,9,DAL,Samaki Walker,Louisville,10.0,445.0,7612.0,2376.0,2089.0,...,4.7,0.6,14.200,0.08900,-2.60,-1.1,0.079793,-4.483371,101.874607,97.406517
956.0,1996,10,IND,Erick Dampier,Mississippi State,16.0,987.0,24003.0,7309.0,7005.0,...,7.1,0.8,52.700,0.10500,-1.30,4.1,0.082994,0.622594,102.671530,103.291084


In [7]:
# TODO: Maybe add toggle button that allows comparing only to players of same position

# group the df_career dataframe by 'Pk' and calculate the average for each relevant stat
df_avg = df_career_na_filled.groupby('Pk')[relevant_stats].mean(numeric_only=True)

df_avg

Player ID,PTS,TRB,AST,WS,WS/48,BPM,VORP,PIE,OFF_RATING,DEF_RATING,NET_RATING
Pk,,,,,,,,,,,
1,10584.814815,4343.777778,2085.592593,58.014815,0.120926,1.429630,23.292593,0.124651,107.487503,106.777783,0.704352
2,7187.888889,3181.740741,1377.740741,35.960222,0.083764,-0.691111,10.307407,0.100532,103.784086,107.382250,-3.207812
3,10229.111111,3584.666667,2222.814815,52.744444,0.116148,1.274074,19.259259,0.117689,107.041839,107.019079,0.020454
4,8261.629630,3193.777778,2149.666667,42.859259,0.094815,-0.620741,13.607407,0.101631,106.379093,107.328083,-0.949534
5,8369.962963,2950.629630,1954.444444,39.886148,0.083815,-0.514815,13.133333,0.097731,105.009115,106.955090,-1.945165
6,4981.259259,1965.925926,977.444444,23.578741,0.080430,-0.666667,6.881481,0.090968,104.194691,106.552246,-2.353500
7,7493.222222,2754.370370,1558.444444,32.992593,0.084407,-1.209630,7.862963,0.095110,105.717086,107.377567,-1.664906
8,5479.666667,2033.259259,1219.111111,22.874074,0.076296,-1.665185,4.966667,0.085489,104.309812,106.343669,-2.033688
9,7624.037037,3266.888889,1507.222222,42.896296,0.095296,-0.533333,14.081481,0.100264,105.066280,106.602401,-1.530061


In [8]:
# for each relevant stat, create a new column in the df_career_na_filled dataframe with the difference between the player's stat and the average for that stat for their draft position
for stat in relevant_stats:
    df_career_na_filled[stat + '_diff'] = df_career_na_filled[stat] - df_career_na_filled['Pk'].map(df_avg[stat])

df_career_na_filled.head(60)

Player ID,Season,Pk,Tm,Player,College,Yrs,G,MP,PTS,TRB,...,TRB_diff,AST_diff,WS_diff,WS/48_diff,BPM_diff,VORP_diff,PIE_diff,OFF_RATING_diff,DEF_RATING_diff,NET_RATING_diff
947.0,1996,1,PHI,Allen Iverson,Georgetown,14.0,914.0,37584.0,24368.0,3394.0,...,-949.777778,3538.407407,40.985185,0.005074,1.770370,26.307407,0.012034,-4.113761,-4.241678,0.142038
948.0,1996,2,TOR,Marcus Camby,UMass,17.0,973.0,28684.0,9262.0,9513.0,...,6331.259259,459.259259,45.639778,0.053236,2.991111,20.992593,0.026252,-0.813788,-4.641551,3.436384
949.0,1996,3,VAN,Shareef Abdur-Rahim,California,12.0,830.0,28882.0,15028.0,6239.0,...,2654.333333,-113.814815,18.455556,0.001852,-0.274074,2.640741,0.008191,-6.239912,-1.175465,-5.043345
950.0,1996,4,MIL,Stephon Marbury,Georgia Tech,13.0,846.0,31891.0,16297.0,2516.0,...,-677.777778,4321.333333,34.640741,0.022185,2.420741,17.092593,0.021741,-2.878029,-2.037657,-0.831081
951.0,1996,5,MIN,Ray Allen,UConn,18.0,1300.0,46344.0,24505.0,5272.0,...,2321.370370,2406.555556,105.213852,0.066185,3.414815,44.466667,0.021744,2.930885,-2.908090,5.829780
952.0,1996,6,BOS,Antoine Walker,Kentucky,12.0,893.0,31531.0,15647.0,6891.0,...,4925.074074,2192.555556,14.521259,-0.022430,0.966667,11.118519,0.015756,-1.796706,-2.045191,0.231327
953.0,1996,7,LAC,Lorenzen Wright,Memphis,13.0,778.0,18535.0,6191.0,4943.0,...,2188.629630,-936.444444,-2.492593,-0.005407,-1.390370,-10.862963,-0.015348,-6.441636,-4.493891,-1.947819
954.0,1996,8,NJN,Kerry Kittles,Villanova,8.0,507.0,16929.0,7165.0,1983.0,...,-50.259259,75.888889,21.925926,0.050704,3.865185,13.033333,0.021012,-2.312377,-5.393768,3.093452
955.0,1996,9,DAL,Samaki Walker,Louisville,10.0,445.0,7612.0,2376.0,2089.0,...,-1177.888889,-1255.222222,-28.696296,-0.006296,-2.066667,-15.181481,-0.020471,-7.659763,-4.727795,-2.953310
956.0,1996,10,IND,Erick Dampier,Mississippi State,16.0,987.0,24003.0,7309.0,7005.0,...,4579.555556,-647.074074,20.044444,0.024741,-0.122222,-5.322222,-0.004984,-1.778714,-3.831300,2.052403


**Calculate ranks for all players above a minimum amount of games played**

This section calculates the ranking of the performance above/below the average player at that draft position for each stat. Some of the stats that are normalized per 48 minutes can be affected by players that only played very little but performed well in this limited time (e.g., a player only ever played two minutes at the end of a blowout but scored four points). To avoid such players being ranked very highly, a minimum amount of games can be set here and all players that do not meet this requirement are excluded from the ranking.

In [9]:
min_games = 82 # TODO: Replace with widget sometime

# create a new column for each relevant stat with the rank of the player's stat for their draft position, only using player with min_games
for stat in relevant_stats:
    if stat == 'DEF_RATING': # lower DEF_RATING is better
        df_career_na_filled[stat + '_rank'] = df_career_na_filled[df_career_na_filled['G'] >= min_games][stat + '_diff'].rank(ascending=True, method='min')
    else:
        df_career_na_filled[stat + '_rank'] = df_career_na_filled[df_career_na_filled['G'] >= min_games][stat + '_diff'].rank(ascending=False, method='min')

**Show Performance of Selected Player**

In [21]:
import ipywidgets as widgets
from IPython.display import display, HTML
from tabulate import tabulate

# create a combobox widget with all 'Player' values
player_widget = widgets.Combobox(
    placeholder='Choose a Player',
    options=df_career_na_filled['Player'].unique().tolist(),
    description='Player:',
    ensure_option=True,
    disabled=False
)

# Define the output area to display additional information
output_player_ranks = widgets.Output()

# Function to update the output area based on the selected player
def on_value_change_player_ranks(change):
    output_player_ranks.clear_output()
    selected_player = change['new']
    with output_player_ranks:
        player_data = df_career_na_filled[df_career_na_filled['Player'] == selected_player]
        display(HTML(f"<h3>Player: {player_data['Player'].values[0]}</h3>"))
        display(HTML(f"<p>Season: {player_data['Season'].values[0]}</p>"))
        display(HTML(f"<p>Draft Pick: {player_data['Pk'].values[0]}</p>"))
        stats_table = [relevant_stats]
        row = []
        for stat in relevant_stats:
            rank = int(player_data[f'{stat}_rank'].values[0])
            actual_stat = round(player_data[stat].values[0], 1)
            diff_stat = round(player_data[f'{stat}_diff'].values[0], 1)
            if diff_stat > 0:
                diff_stat = "+" + str(diff_stat)
            row += f"{rank}. - {actual_stat} ({diff_stat})",
        stats_table.append(row)
        display(HTML(tabulate(stats_table, tablefmt="html")))

# Observe changes in the value of the combobox and call the function
player_widget.observe(on_value_change_player_ranks, names='value')

# Display the widgets
display(player_widget)
display(output_player_ranks)

Combobox(value='', description='Player:', ensure_option=True, options=('Allen Iverson', 'Marcus Camby', 'Share…

Output()

In [22]:
import plotly.express as px
from IPython.display import clear_output

# Filter players based on the minimum games played
filtered_df = df_career_na_filled[df_career_na_filled['G'] >= min_games]

# create a combobox widget with all 'Player' values
player_widget_stat_scatter = widgets.Combobox(
    placeholder='Choose a Player',
    options=df_career_na_filled['Player'].unique().tolist(),
    description='Player:',
    ensure_option=True,
    disabled=False
)

# create a dropdown widget with all relevant stats
stat_dropdown = widgets.Dropdown(
    options=relevant_stats,
    description='Select Stat:',
    disabled=False,
)

# Define the output area to display the scatter plot
output_stat_scatter = widgets.Output()

# Function to update the output area based on the selected stat
def on_value_change_stat_scatter(change):
    clear_output()
    output_stat_scatter.clear_output()
    selected_stat = stat_dropdown.value
    selected_player = player_widget_stat_scatter.value

    display(player_widget_stat_scatter)
    display(stat_dropdown)
    with output_stat_scatter:
        fig = px.scatter(filtered_df, x='Pk', y=f'{selected_stat}_diff', hover_name='Player',
                         hover_data={'Pk': True, f'{selected_stat}_diff': True, 'Player': False, selected_stat: True, f'{selected_stat}_rank': True})
        if selected_player in filtered_df['Player'].values:
            highlighted_player = filtered_df[filtered_df['Player'] == selected_player]
            fig.add_trace(px.scatter(highlighted_player, x='Pk', y=f'{selected_stat}_diff', hover_name='Player',
                                     hover_data={'Pk': True, f'{selected_stat}_diff': True, 'Player': False, selected_stat: True, f'{selected_stat}_rank': True},
                                     color_discrete_sequence=['red']).data[0])
        fig.update_traces(marker=dict(size=12), showlegend=False)
        fig.update_layout(title=f'{selected_stat} Difference vs Draft Pick for Players with at least {min_games} Games Played',
                          xaxis_title='Draft Pick', yaxis_title=f'{selected_stat} Difference')
        fig.show()
    
# Observe changes in the value of the dropdown and call the function
stat_dropdown.observe(on_value_change_stat_scatter, names='value')
player_widget_stat_scatter.observe(on_value_change_stat_scatter, names='value')

# Display the dropdown and the output area
display(player_widget_stat_scatter)
display(stat_dropdown)
display(output_stat_scatter)

Combobox(value='Jayson Tatum', description='Player:', ensure_option=True, options=('Allen Iverson', 'Marcus Ca…

Dropdown(description='Select Stat:', options=('PTS', 'TRB', 'AST', 'WS', 'WS/48', 'BPM', 'VORP', 'PIE', 'OFF_R…